# A SIMPLE EXAMPLE

# KINDS OF PATTERNS

# PATTERN GUARDS

# PATTERN OVERLAPS

# SEALED CLASSES

# THE OPTION TYPE

# PATTERNS EVERYWHERE

# A LARGER EXAMPLE

In [1]:
// layout abstraction in chapter 10

abstract class Element {
  import Element.elem
    
  def contents: Array[String]
  def width: Int =
    if (height == 0) 0 else contents(0).length
  def height: Int = contents.length
  
  def above(that: Element): Element = {
    val this1 = this widen that.width
    val that1 = that widen this.width
    elem(this1.contents ++ that1.contents)
  }

  def beside(that: Element): Element = {
    val this1 = this heighten that.height
    val that1 = that heighten this.height
    elem(
      for ((line1, line2) <- this1.contents zip that1.contents)
      yield line1 + line2)
  }
  
  def widen(w: Int): Element =
    if (w <= width) this
    else {
      val left = elem(' ', (w - width) / 2, height)
      val right = elem(' ', w - width - left.width, height)
      left beside this beside right
    }
  
  def heighten(h: Int): Element =
    if (h <= height) this
    else {
      val top = elem(' ', width, (h - height) / 2)
      val bot = elem(' ', width, h - height - top.height)
      top above this above bot
    }
    
  override def toString = contents mkString "\n"
}

object Element {
  private class ArrayElement(
  val contents: Array[String]
) extends Element

  private class LineElement(s: String) extends Element {
    val contents = Array(s)
    override def width = s.length
    override def height = 1
  }

  private class UniformElement(
    ch: Char,
    override val width: Int,
    override val height: Int
  ) extends Element {
    private val line = ch.toString * width
    def contents = Array.fill(height)(line)
  }

  def elem(contents: Array[String]): Element =
    new ArrayElement(contents)
  def elem(chr: Char, width: Int, height: Int): Element =
    new UniformElement(chr, width, height)
  def elem(line: String): Element =
    new LineElement(line)
}

defined class Element
defined object Element

In [2]:
// expression
sealed abstract class Expr
case class Var(name: String) extends Expr
case class Number(num: Double) extends Expr
case class UnOp(operator: String, arg: Expr) extends Expr
case class BinOp(operator: String, left: Expr, right: Expr) extends Expr

defined class Expr
defined class Var
defined class Number
defined class UnOp
defined class BinOp

In [3]:
// expression formatter

class ExprFormatter {
    private val opGroups = 
        Array(
            Set("|", "||"),
            Set("&", "&&"),
            Set("^"),
            Set("==", "!="),
            Set("<", "<=", ">", ">="),
            Set("+", "-"),
            Set("*", "%"))
    
    private val precedence = {
        val assocs = for {
            i <- 0 until opGroups.length
            op <- opGroups(i)
        } yield op -> i
        assocs.toMap
    }
    
    private val unaryPrecedence = opGroups.length
    private val fractionPrecedence = -1
    
    private def format(e: Expr, enclPrec: Int): Element = {
        import Element.elem
        e match {
            case Var(name) =>
                elem(name)
            case Number(num) =>
                def stripDot(s: String) = 
                    if (s endsWith ".0") s.substring(0, s.length - 2)
                    else s
                elem(stripDot(num.toString))
            case UnOp(op, arg) =>
                elem(op) beside format(arg, unaryPrecedence)
            case BinOp("/", left, right) =>
                val top = format(left, fractionPrecedence)
                val bot = format(right, fractionPrecedence)
                val line = elem('-', top.width max bot.width, 1)
                val frac = top above line above bot
                if (enclPrec != fractionPrecedence) frac
                else elem(" ") beside frac beside elem(" ")
            case BinOp(op, left, right) =>
                val opPrec = precedence(op)
                val l = format(left, opPrec)
                val r = format(right, opPrec + 1)
                val oper = l beside elem(" " + op + " ") beside r
                if (enclPrec <= opPrec) oper
                else elem("(") beside oper beside elem(")")
        }
    }
    
    def format(e: Expr): Element = format(e, 0)
}

defined class ExprFormatter

In [4]:
val f = new ExprFormatter
val e1 = BinOp("*", BinOp("/", Number(1), Number(2)),
               BinOp("+", Var("x"), Number(1)))
val e2 = BinOp("*", BinOp("/", Var("x"), Number(2)),
               BinOp("/", Number(1.5), Var("x")))
val e3 = BinOp("/", e1, e2)

f: ExprFormatter = ammonite.$sess.cmd2$Helper$ExprFormatter@191681f7
e1: BinOp = BinOp(
  "*",
  BinOp("/", Number(1.0), Number(2.0)),
  BinOp("+", Var("x"), Number(1.0))
)
e2: BinOp = BinOp(
  "*",
  BinOp("/", Var("x"), Number(2.0)),
  BinOp("/", Number(1.5), Var("x"))
)
e3: BinOp = BinOp(
  "/",
  BinOp(
    "*",
    BinOp("/", Number(1.0), Number(2.0)),
    BinOp("+", Var("x"), Number(1.0))
  ),
  BinOp(
    "*",
    BinOp("/", Var("x"), Number(2.0)),
    BinOp("/", Number(1.5), Var("x"))
  )
)

In [5]:
for (e <- Array(e1, e2, e3))
    println(f.format(e))

1          
- * (x + 1)
2          
x   1.5
- * ---
2    x 
1          
- * (x + 1)
2          
-----------
  x   1.5  
  - * ---  
  2    x   
